# Forcria
-----------------------------


## Importando bibliotecas

In [4]:
from JogoDeForca import JogoDeForca
import re
import pandas as pd
import random

## Criando o Jogador

In [5]:

class Jogador():
    #instanciando os atributos do jogador
    def __init__(self, tamanho_palavra, content):
        #dicionario com acertos e indices de cada acerto
        self.acertos = {}
        #string que contem todas as letras que foram erros
        self.erros = ""
        #tamanho da palavra a ser advinhada
        self.tamanho_palavra = tamanho_palavra
        #dicionário com a quantidade de palavras que cada letra aparece, de acordo com as palavras possiveis
        self.dict_probabilidades_letra = {}
        #regex que é utilizado para verificar se uma palavra corresponde as informações que já obtemos
        self.regex = "^"+"[a-z]"*tamanho_palavra+"$"
        #lista de palavras possiveis
        self.palavras_possiveis = list(map(str.lower, content))
        #string com letras tentadas
        self.letras_tentadas = ""

    #função responsável por atualizar o regex a partir das informações conhecidas
    def atualiza_regex(self):
        #caso nao tenha tentado nenhuma letra o padrao é [a-z] caso contrário o padrao utilizado para as letras que não sabemos é a negação de todas letras já tentadas
        if self.letras_tentadas == "":
            padrao = "[a-z]"
        else:
            padrao = "[^"+f"{self.letras_tentadas}"+"]"
        lista = [padrao for i in range(self.tamanho_palavra)]
        #adiciona no regex as letras acertadas
        for letra,indices in self.acertos.items():
            for indice in indices:
                lista[indice] = letra
        self.regex = "^"+"".join(lista)+"$"

    #função responsável por atualizar lista de palavras possíveis.
    def atualiza_base_de_palavras(self):
        p_finais = []
        #instancia o regex
        regex = re.compile(self.regex)
        #para cada palavra verifica se é correspondente com o regex, em caso afirmativo adiciona na lista de palavras possiveis
        for palavra in self.palavras_possiveis:
            if regex.search(palavra):
                p_finais.append(palavra)
        return p_finais
    
    #função responsável por determinar a letra a ser jogada
    def escolhe_melhor_jogada(self):
        melhor_letra = ""
        maior_valor = 0
        #para cada letra no dicionário de probabilidades verifica se é a letra com maior probabilidade
        for letra in self.dict_probabilidades_letra:
            #verifica se a letra ainda não foi tentada
            if self.dict_probabilidades_letra[letra] >= maior_valor and letra not in self.letras_tentadas:
                maior_valor = self.dict_probabilidades_letra[letra]
                melhor_letra = letra
        self.letras_tentadas += melhor_letra
        #retorna a letra com maior probabilidade
        return melhor_letra

    #função que atualiza o dicionário de acertos
    def atualiza_dict_acertos(self, letra, lista_acertos):
        #adiciona no dicionario caso a lista de indices acertados seja diferente de []
        if lista_acertos != []:
            self.acertos[letra] = lista_acertos
        #caso contrário adiciona na string de erros
        else:
            self.erros += letra
    
    #atualiza a probabilidade de cada letra
    def atualiza_probabilidades(self):
        alfabeto = "abcdefghijklmnopqrstuvwxyz"
        dict = {letra : 0 for letra in alfabeto}
        #para cada letra do alfabeto adiciona um no seu contador caso ela apareça em uma palavra
        for palavra in self.palavras_possiveis:
            letras_ja_adicionadas = ""
            for letra in palavra.lower():
                if letra not in letras_ja_adicionadas:
                    dict[letra] += 1
                    letras_ja_adicionadas += letra
        return dict
    
    #função que realiza a jogada toda
    def jogada(self, letra, lista_acertos):
        #atualiza o dicionário de acertos
        self.atualiza_dict_acertos(letra, lista_acertos)
        #atualiza o regex com base no dicionario atualizado
        self.atualiza_regex()
        #atualiza as palavras possiveis com base no regex atualizado
        self.palavras_possiveis = self.atualiza_base_de_palavras()
        #atualiza a probabilidade de cada letra com base na lista de palavras
        self.dict_probabilidades_letra = self.atualiza_probabilidades()
        #escolhe a melhor jogada em relação as probabilidades de cada letra
        return self.escolhe_melhor_jogada()

    


### Lógica de decisão do Jogador : 
_____________

No processo de escolha de letras para o palpite, utilizamos a lógica de uma árvore de decisão, que favorece a seleção da letra que apresenta maior frequência dentre as palavras elegíveis para chute, levando em consideração a presença unitária da letra em cada uma das palavras. Com isso, o jogador sempre opta por chutar a letra que possui maior probabilidade de estar presente na palavra a ser adivinhada.

Essa estratégia visa maximizar as chances de acerto e minimizar as possibilidades de perda de vidas, possibilitando a adivinhação de mais letras e suas respectivas posições. Além disso, a seleção da letra com maior frequência nos permite eliminar as palavras que não possuem tal letra, o que ajuda a aproximar ainda mais o palpite do resultado correto.

## Instanciando o Jogador
E realizando as partidas

In [6]:
#inicializando a quantidade de vitorias e derrotas e a lista de derrotas
qtd_vitorias = 0
qtd_derrotas = 0
lista_derroatas = []
#cria o jogo 100 vezes
for i in range(100):
    #inicializa o jogo de forca (Desenvolvido pelo professor)
    jogo = JogoDeForca()
    tamanho_palavra = jogo.novo_jogo()
    #var inicial recebe a palavra esperada
    inicial = jogo.palavra
    #inicializa o jogador
    jogador = Jogador(tamanho_palavra, jogo.content)
    letra = ""
    #lista de acertos que receberá o feedback do "juiz"
    lista_acertos = []
    terminou = False
    # loop que realiza as jogadas enquanto a quantidade de vidas é maior ou igual a 1.
    while terminou == False and jogo.vidas >= 1:
        #letra chutada pelo algortimo
        letra = jogador.jogada(letra,lista_acertos)
        #caso a lista de palavras possiveis só tenha uma palavra
        if len(set(jogador.palavras_possiveis)) == 1: 
            #chuta a palavra
            chute = jogador.palavras_possiveis[0]
            j = jogo.tentar_palavra(chute)
            #caso tenha acertado a palavra
            if j:
                #caso tenha ganhado aumenta a quantidade de vitórias
                qtd_vitorias += 1
            #caso tenha errado a palavra
            else:
                terminou = True
                chute = jogador.palavras_possiveis
                qtd_derrotas += 1
                lista_derroatas.append([inicial,chute])
            terminou = True
            break
        #caso contrário
        else:
            #caso o tamanho da lista de palavras possiveis seja maior que 1, o algortimo tenta uma letra.
            lista_acertos = jogo.tentar_letra(letra)
            #caso tenha errado a letra e as vidas tenham acabado
            if lista_acertos == False:
                terminou = True
                chute = jogador.palavras_possiveis
                qtd_derrotas += 1
                lista_derroatas.append([inicial,chute])
    
print(f"Quantidade de vitorias: {qtd_vitorias}")
print(f"Quantidade de derrotas: {qtd_derrotas}")
#(formato: [palavra, lista_de_palavras_possíveis]) 
print(f"Lista com os chutes errados: {lista_derroatas}")

dict = {"erros": lista_derroatas}
df = pd.DataFrame(dict)
df.to_csv("erros.csv")

Quantidade de vitorias: 95
Quantidade de derrotas: 5
Lista com os chutes errados: [['adubava', ['abusava', 'acenava', 'acusava', 'adubava', 'afetava', 'agucava', 'ajudava', 'anexava', 'apegava', 'apenava', 'asseava', 'autuava', 'azedava']], ['labora', ['labora', 'majora']], ['cobrei', ['cobrei', 'honrei', 'logrei']], ['rico', ['rico', 'rido', 'rifo', 'rijo']], ['sara', ['dara', 'fara', 'iara', 'sara', 'tara']]]


## Resultados


Ao utilizar a técnica descrita, é possível alcançar uma acurácia superior a 90% nas partidas de forca, tendo em vista que a estratégia favorece a escolha da letra com maior frequência entre as palavras elegíveis para chute. Entretanto, é importante destacar que essa abordagem pode apresentar dificuldades em palavras pequenas com grande variabilidade nas letras restantes, uma vez que a estratégia pode não ser capaz de eliminar palavras o suficiente para chegar ao resultado correto como caso da palavra 'rico' que tem as seguintes variações : 'rico', 'rido', 'rifo', 'rijo'.

Nesse sentido, é necessário considerar que a eficácia da técnica está relacionada à disponibilidade de informações precisas sobre as palavras elegíveis para chute, o que pode ser limitado em algumas situações. Além disso, é importante ressaltar que outros fatores, como a complexidade e a diversidade do vocabulário utilizado nas palavras a serem adivinhadas, também podem influenciar na acurácia da estratégia.

Portanto, apesar de apresentar resultados promissores, é fundamental avaliar cuidadosamente as limitações e as possíveis variações do método, a fim de garantir uma abordagem mais robusta e eficaz para a adivinhação de palavras no jogo de forca.